In [128]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import urllib
import requests
import json
import numpy as np
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

# 프로그래머스

In [86]:
# 초기 세팅
collection_time = datetime.now().strftime('%Y-%m-%d-%H')

session = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0'}
session.headers.update(headers)

url_list = []
i=1
previous_response = None

while True:
    # url 불러오기
    url = f'https://career.programmers.co.kr/api/job_positions?order=recent&page={i}&job_category_ids[]=1&job_category_ids[]=4&job_category_ids[]=25&job_category_ids[]=2&job_category_ids[]=3&job_category_ids[]=5&job_category_ids[]=11&job_category_ids[]=12&job_category_ids[]=92&job_category_ids[]=7&job_category_ids[]=16&job_category_ids[]=20&job_category_ids[]=9&job_category_ids[]=18&job_category_ids[]=59&job_category_ids[]=22&job_category_ids[]=26&job_category_ids[]=27&job_category_ids[]=10&job_category_ids[]=13&job_category_ids[]=17&job_category_ids[]=6&job_category_ids[]=125&job_category_ids[]=126&job_category_ids[]=127&job_category_ids[]=128&job_category_ids[]=129&job_category_ids[]=130&job_category_ids[]=158'

    res = requests.get(url,headers= headers)
    res_list = json.loads(res.text)

    # 이전 응답과 비교하여 같으면 정지
    if res_list == previous_response:
        print("url 수집 종료")
        break

    # 현재 응답을 이전 응답으로 저장
    previous_response = res_list

    for j in range(len(res_list['jobPositions'])):
        url_list.append('https://career.programmers.co.kr/'+res_list['jobPositions'][j]['url'])

    i += 1

def fetch_job_data(href):
    try:
        id = re.findall(r'\d+',href)
        url = f'https://career.programmers.co.kr/api/job_positions/{id[0]}'
        res = session.get(url)
        res.raise_for_status()
        pro_json = res.json()
        id = str(pro_json['jobPosition']['id'])
        return id, json.dumps(pro_json, ensure_ascii=False), collection_time
    except Exception as e:
        print(f"Error processing {href}: {e}")
        return None

# 병렬 처리
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(fetch_job_data, href): href for href in url_list}
    pro_data = []
    for future in as_completed(future_to_url):
        result = future.result()
        if result:
            pro_data.append(result)

# json 원본 파일 저장
pdf = pd.DataFrame(pro_data, columns=['id', 'json', 'collection_time'])
raw_json_file = f'programmers_json_{collection_time}.csv'
pdf.to_csv(raw_json_file, index=False, encoding='utf-8-sig')
print(f"Raw JSON data saved to {raw_json_file}")
pdf.info()


url 수집 종료
Error processing https://career.programmers.co.kr//job_positions/15261: 401 Client Error: Unauthorized for url: https://career.programmers.co.kr/api/job_positions/15261
Error processing https://career.programmers.co.kr//job_positions/15258: 401 Client Error: Unauthorized for url: https://career.programmers.co.kr/api/job_positions/15258
Error processing https://career.programmers.co.kr//job_positions/15250: 401 Client Error: Unauthorized for url: https://career.programmers.co.kr/api/job_positions/15250
Error processing https://career.programmers.co.kr//job_positions/15249: 401 Client Error: Unauthorized for url: https://career.programmers.co.kr/api/job_positions/15249
Error processing https://career.programmers.co.kr//job_positions/15521: 401 Client Error: Unauthorized for url: https://career.programmers.co.kr/api/job_positions/15521
Raw JSON data saved to programmers_json_2025-01-17-16.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (t

In [ ]:

# 직군 매핑 딕셔너리
job_dic = {
    1: "서버/백엔드", 4: "프론트엔드", 25: "웹 풀스택", 2: "안드로이드", 3: "iOS",
    5: "머신러닝", 11: "인공지능(AI)", 12: "데이터 엔지니어링", 92: "DBA",
    7: "모바일 게임", 16: "게임 클라이언트", 20: "게임 서버", 9: "시스템/네트워크",
    18: "시스템 소프트웨어", 59: "데브옵스", 22: "인터넷 보안", 26: "임베디드 소프트웨어",
    27: "로보틱스 미들웨어", 10: "QA", 13: "사물인터넷(IoT)", 17: "응용 프로그램",
    6: "블록체인", 125: "개발PM", 126: "웹 퍼블리싱", 127: "크로스 플랫폼",
    128: "VR/AR/3D", 129: "ERP", 130: "그래픽스", 158: "데이터 분석"
}

# 초기 데이터 로드
raw_file = f'programmers_json_{collection_time}.csv'
pdf = pd.read_csv(raw_file, encoding='utf-8-sig')

# 데이터 초기화
data = {
    'json': [], 'id': [], 'url': [], 'career': [], 'title': [], 'location': [], 
    'Category': [], 'technicalTags': [], 'countryCode': [],
    'description': [], 'requirement': [], 'preferredExperience': [], 
    'CompanyName': [], 'collectionTime': []
}

# JSON 데이터 파싱 및 처리
for job in pdf['json']:
    job = json.loads(job)
    job_position = job.get('jobPosition')
    if not job_position:
        print("Skipping job without 'jobPosition':", job)
        continue

    # 데이터 추가
    data['json'].append(job)
    data['id'].append(job_position.get('id'))
    data['url'].append('https://career.programmers.co.kr' + job_position.get('url', ''))
    data['career'].append(job_position.get('career', ''))
    data['title'].append(job_position.get('title', ''))
    data['location'].append(job_position.get('address', ''))
    data['Category'].append(", ".join(job_dic[j] for j in job_position.get('jobCategoryIds', [])))
    data['technicalTags'].append(", ".join(tag['name'] for tag in job_position.get('technicalTags', [])))
    data['countryCode'].append(job_position.get('countryCode', ''))
    data['description'].append(job_position.get('description', ''))
    data['requirement'].append(job_position.get('requirement', ''))
    data['preferredExperience'].append(job_position.get('preferredExperience', ''))
    data['CompanyName'].append(job_position['company'].get('name', ''))
    data['collectionTime'].append(collection_time)

# 데이터프레임 생성
pro_df = pd.DataFrame(data)

# 경력 정보에서 min_career와 max_career를 추출하는 함수
def parse_career(career):
    if "경력 무관" in career or not career:
        return 0, 100
    if "신입" in career:
        return 0, 1
    match = re.search(r"(\d+)\s*~\s*(\d+)", career)
    if match:
        return map(int, match.groups())
    match = re.search(r"(\d+)", career)
    if match:
        year = int(match.group(1))
        return year, year
    return 0, 100

# min_career와 max_career 계산
pro_df["min_career"], pro_df["max_career"] = zip(*pro_df["career"].apply(parse_career))

# 주요 지역 추출 함수
def extract_region(address):
    if not isinstance(address, str) or pd.isna(address):
        return '기타'
    patterns = {
        '서울': r'(서울|서울특별시)', '경기': r'(경기|경기도)', '인천': r'(인천|인천광역시)',
        '강원': r'(강원|강원도)', '부산': r'(부산|부산광역시)', '대구': r'(대구|대구광역시)',
        '대전': r'(대전|대전광역시)', '광주': r'(광주|광주광역시)', '울산': r'(울산|울산광역시)',
        '세종': r'(세종|세종특별자치시)', '제주': r'(제주|제주특별자치도)', '전북': r'(전북|전라북도)',
        '전남': r'(전남|전라남도)', '충북': r'(충북|충청북도)', '충남': r'(충남|충청남도)',
        '경북': r'(경북|경상북도)', '경남': r'(경남|경상남도)', '재택': r'(재택|재택근무|재택 근무)'
    }
    for region, pattern in patterns.items():
        if re.search(pattern, address):
            return region
    return '기타'

# 지역 정보 추출
pro_df['location'] = pro_df['location'].apply(extract_region)

# 중복 제거
pro_df.drop_duplicates(inplace=True)

# HTML 파싱 및 데이터 정리 함수
def parse_html_to_columns(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    
    # 데이터 분류
    description = []
    preferred_experience = []
    requirements = []

    for tag in soup.find_all(["h3", "h2", 'strong', "ul", "li"]):
        tag_text = tag.get_text(strip=True)

        # 직무 설명
        if ("직무 설명" in tag_text or 
            "포지션 소개" in tag_text or
            "주요업무" in tag_text or
            "주요 업무" in tag_text or
            "[주요업무]" in tag_text or
            "주요 소개" in tag_text or
            "조직소개" in tag_text or
            "직무 소개" in tag_text or
            "직무소개" in tag_text):
            next_ul = tag.find_next("ul")
            if next_ul:  # ul 태그가 있는 경우만 처리
                description.append(next_ul.get_text(strip=True))

        # 자격 요건
        elif ("자격 요건" in tag_text or
              "자격요건" in tag_text or
              "자격 조건" in tag_text or
              "자격조건" in tag_text or
              "[자격 요건]" in tag_text or
              "[자격요건]" in tag_text or
              "[자격 조건]" in tag_text or
              "[자격조건]" in tag_text or
              "지원자격" in tag_text or
              "지원 자격" in tag_text or
              "[지원자격]" in tag_text or
              "[지원 자격]" in tag_text):
            next_ul = tag.find_next("ul")
            if next_ul:
                requirements.append(next_ul.get_text(strip=True))
                
        # 우대 사항
        elif ("우대 사항" in tag_text or 
              "우대사항" in tag_text or
              "우대 조건" in tag_text or 
              "우대조건" in tag_text or
              "[우대사항]" in tag_text or 
              "[우대 사항]" in tag_text or
              "[우대조건]" in tag_text or
              "[우대 조건]" in tag_text):
            next_ul = tag.find_next("ul")
            if next_ul:
                preferred_experience.append(next_ul.get_text(strip=True))


    return {
        "description": " ".join(description),
        "requirement": " ".join(requirements),
        "preferredExperience": " ".join(preferred_experience)
    }

# 데이터 처리
parsed_data = pro_df["description"].apply(parse_html_to_columns)

# 결과를 새로운 데이터프레임으로 정리
parsed_df = pd.DataFrame(parsed_data.tolist())
parsed_df['id'] = pro_df['id']  # id를 유지

for col in ["description", "requirement", "preferredExperience"]:
    parsed_df[col] = parsed_df[col].replace('', np.nan)  # 빈 문자열을 NaN으로 변환
    parsed_df[col] = parsed_df[col].fillna(pro_df[col])  # NaN을 원본 데이터로 채우기

# 태그 제거 함수 정의
def remove_html_tags(text):
    if pd.isna(text):  # NaN 처리
        return text
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text(strip=True)

# 각 열에 HTML 태그 제거 적용
columns_to_clean = ["description", "requirement", "preferredExperience"]
for col in columns_to_clean:
    parsed_df[col] = parsed_df[col].apply(remove_html_tags)

# requirement 또는 preferredExperience 열에서 NaN 값 또는 빈 문자열인 행 삭제
parsed_df = parsed_df.dropna(subset=["requirement", "preferredExperience"])  # NaN 값 제거
parsed_df = parsed_df[(parsed_df["requirement"] != '') & (parsed_df["preferredExperience"] != '')]  # 빈 문자열 제거

# 원본 데이터에서 requirement, preferredExperience, description 열 제거
columns_to_remove = ["description", "requirement", "preferredExperience", "career"]
pro_df = pro_df.drop(columns=[col for col in columns_to_remove if col in pro_df.columns], errors="ignore")

# 나머지 열과 parsed_df 병합
final_df = pd.merge(pro_df, parsed_df, on='id')

# 정리된 데이터 저장
output_path = f"programmers_df_{collection_time}.csv"
final_df.to_csv(output_path, index=False, encoding="utf-8-sig")

# 데이터프레임 정보 확인
final_df.info()


C:\Users\Anichan\AppData\Local\Temp\ipykernel_15324\476961097.py:93: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, "html.parser")
C:\Users\Anichan\AppData\Local\Temp\ipykernel_15324\476961097.py:168: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697 entries, 0 to 696
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   697 non-null    int64 
 1   url                  697 non-null    object
 2   title                697 non-null    object
 3   location             697 non-null    object
 4   Category             697 non-null    object
 5   technicalTags        697 non-null    object
 6   countryCode          697 non-null    object
 7   CompanyName          697 non-null    object
 8   collectionTime       697 non-null    object
 9   min_career           697 non-null    int64 
 10  max_career           697 non-null    int64 
 11  description          697 non-null    object
 12  requirement          697 non-null    object
 13  preferredExperience  697 non-null    object
dtypes: int64(3), object(11)
memory usage: 76.4+ KB


# 원티드

In [1]:

SCROLL_PAUSE_TIME = 4
WANTED_URL = 'https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=all'

global start_time
path = 'C:/Users/Anichan/Documents/공부자료/24.12 아시아 경제/work/project LLM/크롤링/'
set_path = f'{path}wanted_wdlist_href.csv'


def chrome_driver():
    # 크롬드라이버 위치
    path1 = r"C:/Users/Anichan/Downloads/chromedriver-win64/chromedriver.exe"

    s = Service(path1)
    driver = webdriver.Chrome(service=s)

    driver.get(WANTED_URL)
    time.sleep(3)

    print(f'driver wanted loading 완료 : {time.time() - start_time: .5f}')

    last_height = driver.execute_script('return document.body.scrollHeight')
    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    try:
        print(f'wanted page scroll 시작 : {time.time() - start_time: .5f}')

        retry = 0
        while True:
            # 페이지 끝까지 스크롤
            actions.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

            # 새로운 페이지 높이 확인
            new_height = driver.execute_script('return document.body.scrollHeight')
            if new_height == last_height:
                print(f'wanted page retry : {retry}')
                retry += 1
                if retry >= 5:
                    break
            else:
                retry = 0
                last_height = new_height

        print('모든 컨텐츠가 로드되었습니다.')
        print(f'wanted page scroll 종료 : {time.time() - start_time: .5f}')

    except Exception as e:
        print(f'Error : {time.time() - start_time: .5f}')
        print(e)
        return False

    href_list = Extract_href(driver)

    set_href(href_list)

def Extract_href(driver):
    print(f'href element extraction 시작 : {time.time() - start_time: .5f}')

    href_list = []

    job_cards = driver.find_elements(By.CSS_SELECTOR, 'ul[data-cy="job-list"] li a')
    for job_card in job_cards:
        href = job_card.get_attribute('href')

        if href and href not in href_list:
            href_list.append(href)

    print(f'href element extraction 종료 : {time.time() - start_time: .5f}')
    print(f'총 href 개수 : {len(href_list)}')
    print(f'href_list sample : \n {href_list[:5]}')

    return href_list

def set_href(href_list):
    print(f'set href 시작 : {time.time() - start_time: .5f}')

    href_df = pd.DataFrame(href_list, columns=['href'])
    href_df.to_csv(set_path, index=False)

    print(f'set href 종료 : {time.time() - start_time: .5f}, 저장 위치 : {set_path}')

if __name__ == "__main__":
    start_time = time.time()
    print(f'프로그램 시작 : {time.time() - start_time: .5f}')
    chrome_driver()
    print(f'프로그램 종료 : {time.time() - start_time: .5f}')

프로그램 시작 :  0.00000
driver wanted loading 완료 :  8.39725
wanted page scroll 시작 :  8.40915
wanted page retry : 0
wanted page retry : 1
wanted page retry : 2
wanted page retry : 3
wanted page retry : 4
모든 컨텐츠가 로드되었습니다.
wanted page scroll 종료 :  390.61141
href element extraction 시작 :  390.61141
href element extraction 종료 :  403.25669
총 href 개수 : 3151
href_list sample : 
 ['https://www.wanted.co.kr/wd/235498', 'https://www.wanted.co.kr/wd/183944', 'https://www.wanted.co.kr/wd/255353', 'https://www.wanted.co.kr/wd/183943', 'https://www.wanted.co.kr/wd/261274']
set href 시작 :  403.25669
set href 종료 :  403.28056, 저장 위치 : C:/Users/Anichan/Documents/공부자료/24.12 아시아 경제/work/project LLM/크롤링/wanted_wdlist_href.csv
프로그램 종료 :  403.28056


In [72]:
# json 원본 파일 생성 및 전처리 파일 생성

file_path = 'wanted_wdlist_href.csv'
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
    exit()

collection_time = datetime.now().strftime('%Y-%m-%d-%H')

session = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0'}
session.headers.update(headers)

def fetch_job_data(href):
    try:
        wid = re.findall(r'\d+', href)[-1]
        url = f'https://www.wanted.co.kr/api/chaos/jobs/v3/{wid}/details?1736076088092='
        res = session.get(url)
        res.raise_for_status()
        wjob_json = res.json()
        id = str(wjob_json['job']['id'])
        return id, json.dumps(wjob_json, ensure_ascii=False), collection_time
    except Exception as e:
        print(f"Error processing {href}: {e}")
        return None

# 병렬 처리
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(fetch_job_data, href): href for href in df['href']}
    wjob_data = []
    for future in as_completed(future_to_url):
        result = future.result()
        if result:
            wjob_data.append(result)

# json 원본 파일 저장
wdf = pd.DataFrame(wjob_data, columns=['id', 'json', 'collection_time'])
raw_json_file = f'wanted_json_{collection_time}.csv'
wdf.to_csv(raw_json_file, index=False, encoding='utf-8-sig')
print(f"Raw JSON data saved to {raw_json_file}")

def extract_job_data(job):
    if 'job' not in job:
        print("Skipping job without 'job':", job)
        return None

    wjob_position = job['job']['detail']
    job_data = {
        'json': job,
        'id': job['job']['id'],
        'url': f"https://www.wanted.co.kr/wd/{job['job']['id']}",
        'title': wjob_position['position'],
        'location': job['job']['address']['location'],
        'Category': ", ".join([tag['text'] for tag in job['job']['category_tag']['child_tags']]) if job['job']['category_tag']['child_tags'] else None,
        'technicalTags': ", ".join([tag['text'] for tag in job['job']['skill_tags']]) if job['job']['skill_tags'] else None,
        'countryCode': job['job']['address']['country_code'],
        'CompanyName': job['job']['company']['name'],
        'min_career': job['job']['annual_from'],
        'max_career': job['job']['annual_to'],
        'description': wjob_position['main_tasks'],
        'requirement': wjob_position['requirements'],
        'preferredExperience': wjob_position['preferred_points'],
        'collectionTime': collection_time
    }
    return job_data

job_data_list = [extract_job_data(json.loads(job)) for job in wdf['json'] if json.loads(job) is not None]
wanted_df = pd.DataFrame(job_data_list)

wanted_df['countryCode'] = wanted_df['countryCode'].str.upper()
wanted_df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

processed_file = f'wanted_df_{collection_time}.csv'
wanted_df.to_csv(processed_file, index=False, encoding='utf-8-sig')
print(f"Processed data saved to {processed_file}")

wanted_df.info()


Error processing https://www.wanted.co.kr/wd/259709: 503 Server Error: Service Unavailable for url: https://www.wanted.co.kr/api/chaos/jobs/v3/259709/details?1736076088092=
Error processing https://www.wanted.co.kr/wd/244073: 503 Server Error: Service Unavailable for url: https://www.wanted.co.kr/api/chaos/jobs/v3/244073/details?1736076088092=
Raw JSON data saved to wanted_json_2025-01-17-16.csv
Processed data saved to wanted_df_2025-01-17-16.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3149 entries, 0 to 3148
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   3149 non-null   int64 
 1   url                  3149 non-null   object
 2   title                3149 non-null   object
 3   location             3149 non-null   object
 4   Category             3149 non-null   object
 5   technicalTags        1590 non-null   object
 6   countryCode          3149 non-null   object
 

# csv 병합

In [88]:
# 파일 읽기
programmers_df = pd.read_csv(f'programmers_df_{collection_time}.csv')
wanted_df = pd.read_csv(f'wanted_df_{collection_time}.csv')

# 데이터프레임 병합
merged_df = pd.concat([programmers_df, wanted_df], axis=0, ignore_index=True)

merged_df.to_csv(f'merged_df_{collection_time}.csv', index=False, encoding='utf-8-sig')

In [90]:
test = merged_df.copy()

In [ ]:
test

In [125]:
test = pd.read_csv('merged_df_2025-01-17-16.csv')  # 실제 파일 이름으로 변경해야 합니다

# 치환 딕셔너리
replace_dict = {
    "PM": ["개발PM", "개발 매니저", "프로덕트 매니저"],
    "데이터 직무": ["DBA", "데이터 엔지니어링", "데이터 엔지니어", "빅데이터 엔지니어"],
    "백엔드": ["서버/백엔드", "서버 개발자", "웹 개발자"],
    "인프라 엔지니어": ["데브옵스", "인터넷 보안", "DevOps / 시스템 관리자", "보안 엔지니어", "네트워크 관리자", "소프트웨어 엔지니어"],
    "앱 개발자": ["iOS", "안드로이드", "안드로이드 개발자", "iOS 개발자", "크로스플랫폼 앱 개발자"],
    "게임": ["모바일 게임", "게임 클라이언트", "게임 서버"],
    "AI": ["머신러닝", "인공지능(AI)", "머신러닝 엔지니어", "음성 엔지니어"],
    "임베디드": ["사물인터넷(IoT)", "로보틱스 미들웨어", "임베디드 소프트웨어", "하드웨어 엔지니어", "임베디드 개발자", "시스템 소프트웨어"],
    "프론트 엔드": ["웹 퍼블리싱", "웹 퍼블리셔", "프론트엔드 개발자", "그래픽스", "그래픽스 엔지니어", "웹 풀스택", "크로스 플랫폼"],
    "QA": ["QA", "테스트 엔지니어"],
    "데이터 분석": ["데이터 분석", "BI 엔지니어", "데이터 사이언티스트"],
    "VR": ["VR/AR/3D", "VR 엔지니어"],
    "시스템": ["시스템/네트워크", "응용 프로그램", "시스템"],
    "블록체인": ["블록체인", "블록체인 플랫폼 엔지니어"],
    "ERP": ["ERP전문가"],
    "언어별 개발자": ["파이썬 개발자", "자바 개발자", "C", "C++ 개발자", ".NET 개발자", "Node.js 개발자", "PHP 개발자"],
    "삭제": ["Chief Information Officer", "Chief Technology Officer", "CIO", "CTO", "루비온레일즈 개발자", "기술지원", "영상"]
}

def process_categories(categories):
    if pd.isna(categories):  # NaN 값 처리
        return None

    category_list = categories.split(", ")
    processed = set()
    
    # 특별한 경우(웹 풀스택): 백엔드와 프론트 엔드 모두 추가
    if "웹 풀스택" in category_list:
        processed.add("백엔드")
        processed.add("프론트 엔드")
        category_list.remove("웹 풀스택")
    
    for key, values in replace_dict.items():
        for value in values:
            if value in category_list:
                if key == "삭제":  # 삭제에 해당하면 제외
                    category_list.remove(value)
                else:
                    processed.add(key)
                    category_list.remove(value)
    
    if not processed:  # 처리된 카테고리가 없으면 None 반환
        return None

    return ", ".join(processed)

# Category 열 처리
test["Category_test"] = test["Category"].apply(process_categories)

# 빈 값이 된 행 삭제
test = test.dropna(subset=["Category_test"])

# # 결과 저장
# output_path = "processed_merge_df.csv"
# test.to_csv(output_path, index=False, encoding="utf-8-sig")


In [126]:
test[test['Category'] == '웹 풀스택'][['Category_test', 'Category']]

,Category_test,Category
4,"백엔드, 프론트 엔드",웹 풀스택
59,"백엔드, 프론트 엔드",웹 풀스택
75,"백엔드, 프론트 엔드",웹 풀스택
100,"백엔드, 프론트 엔드",웹 풀스택
348,"백엔드, 프론트 엔드",웹 풀스택
380,"백엔드, 프론트 엔드",웹 풀스택
515,"백엔드, 프론트 엔드",웹 풀스택
525,"백엔드, 프론트 엔드",웹 풀스택
549,"백엔드, 프론트 엔드",웹 풀스택
556,"백엔드, 프론트 엔드",웹 풀스택


In [124]:
# '웹 풀스택'이 포함된 행을 필터링
filtered = test[test["Category"].apply(lambda x: "웹 풀스택" in x.split(", ") if pd.notna(x) else False)]

# 결과 출력
print(filtered[["Category", "Category_test"]])


                 Category       Category_test
3           서버/백엔드, 웹 풀스택         백엔드, 프론트 엔드
4                   웹 풀스택         백엔드, 프론트 엔드
21   서버/백엔드, 프론트엔드, 웹 풀스택         백엔드, 프론트 엔드
28   서버/백엔드, 프론트엔드, 웹 풀스택         백엔드, 프론트 엔드
34           프론트엔드, 웹 풀스택         백엔드, 프론트 엔드
..                    ...                 ...
676          프론트엔드, 웹 풀스택         백엔드, 프론트 엔드
677     웹 풀스택, 안드로이드, iOS  백엔드, 앱 개발자, 프론트 엔드
687  서버/백엔드, 프론트엔드, 웹 풀스택         백엔드, 프론트 엔드
688         서버/백엔드, 웹 풀스택         백엔드, 프론트 엔드
696          프론트엔드, 웹 풀스택         백엔드, 프론트 엔드

[140 rows x 2 columns]


In [127]:
import pandas as pd

# 데이터프레임 로드
# df = pd.read_csv('merged_df.csv')  # 실제 파일 이름으로 변경해야 합니다

# Category 열에서 유니크 값 추출
unique_categories = set()

def extract_unique_categories(categories):
    if pd.notna(categories):
        for category in categories.split(','):
            unique_categories.add(category.strip())

# 각 행의 Category 열에서 유니크 값 추출
test['Category_test'].apply(extract_unique_categories)

# 유니크 값 목록 정렬
unique_categories = sorted(unique_categories)

print(len(unique_categories))

# 결과 출력
for i in unique_categories:
    print(i)


16
AI
ERP
PM
QA
VR
게임
데이터 분석
데이터 직무
백엔드
블록체인
시스템
앱 개발자
언어별 개발자
인프라 엔지니어
임베디드
프론트 엔드


In [94]:
import pandas as pd

# 데이터프레임 로드
# df = pd.read_csv('merged_df.csv')  # 실제 파일 이름으로 변경해야 합니다

# Category 열에서 유니크 값 추출
unique_categories = set()

def extract_unique_categories(categories):
    if pd.notna(categories):
        for category in categories.split(','):
            unique_categories.add(category.strip())

# 각 행의 Category 열에서 유니크 값 추출
test['Category'].apply(extract_unique_categories)

# 유니크 값 목록 정렬
unique_categories = sorted(unique_categories)

print(len(unique_categories))

# 결과 출력
for i in unique_categories:
    print(i)


16
AI
ERP
PM
QA
VR
게임
데이터 분석
데이터 직무
백엔드
블록체인
시스템
앱 개발자
언어별 개발자
인프라 엔지니어
임베디드
프론트 엔드


In [ ]:
test.to_csv(f'merged_df_2025-01-17-16_test.csv', index=False, encoding='utf-8-sig')

In [137]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   3799 non-null   int64 
 1   url                  3799 non-null   object
 2   title                3799 non-null   object
 3   location             3799 non-null   object
 4   jobCategory          3799 non-null   object
 5   technicalTags        2093 non-null   object
 6   countryCode          3799 non-null   object
 7   CompanyName          3799 non-null   object
 8   min_career           3799 non-null   int64 
 9   max_career           3799 non-null   int64 
 10  description          3799 non-null   object
 11  requirement          3799 non-null   object
 12  preferredExperience  3754 non-null   object
dtypes: int64(3), object(10)
memory usage: 386.0+ KB


# 테스트

In [ ]:
# 크롬드라이버 위치
path = r"C:/Users/Anichan/Downloads/chromedriver-win64/chromedriver.exe"

# 크롤링할 url
url = 'https://career.programmers.co.kr/job'

s = Service(path)
driver = webdriver.Chrome(service=s)

driver.get(url)

# 직무 설정
btn_ex_path = '//*[@id="search-form"]/div[2]/div[1]/button'
driver.find_element(By.XPATH,btn_ex_path ).click()
time.sleep(1)

# 체크박스 전부 클릭
for i in range(1,30):
    btn_ex_path = f'//*[@id="search-form"]/div[2]/div[1]/div/ul/li[{i}]/label/input'
    driver.find_element(By.XPATH,btn_ex_path ).click()
    time.sleep(1)

for j in range(1)

# # 검색 버튼 클릭
# btn_ex_path2 = '//*[@id="searchForm"]/div/div[8]/button'
# driver.find_element(By.XPATH,btn_ex_path2).click()
# time.sleep(3)


In [ ]:
temp = driver.find_element(By.XPATH, t_path).get_attribute('value')
temp

temp = driver.find_element(By.CLASS_NAME, 'job_category_label')
temp

In [210]:
headers = {'User-Agent': 'Mozilla/5.0'}

url = url_list[0]

# GET 요청
response = requests.get(url, headers=headers)

# BeautifulSoup으로 HTML 파싱
soup = BeautifulSoup(response.text, "html.parser")
soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="프로그래머스" itemprop="title name" name="twitter:title" property="og:title"/>
<meta content="프로그래머스" itemprop="title name" name="og:title" property="og:title"/>
<meta content="SW개발자를 위한 평가, 교육, 채용까지 Total Solution을 제공하는 개발자 성장을 위한 베이스캠프" itemprop="description" name="twitter:description" property="og:description"/>
<meta content="SW개발자를 위한 평가, 교육, 채용까지 Total Solution을 제공하는 개발자 성장을 위한 베이스캠프" itemprop="description" name="og:description" property="og:description"/>
<meta content="https://career.programmers.co.kr/assets/img-meta-programmers-411e94bf29153dc31004168e6cd500279b1a531a23689303755e51971dee4526.png" itemprop="image primaryImageOfPage" name="twitter:image" property="og:image"/>
<meta content="https://career.programmers.co.kr/assets/img-meta-programmers-411e9

In [332]:
df.columns

Index(['jobPosition.id', 'jobPosition.address', 'jobPosition.career',
       'jobPosition.careerRange', 'jobPosition.companyId',
       'jobPosition.jobType', 'jobPosition.maxSalary', 'jobPosition.minSalary',
       'jobPosition.personalized', 'jobPosition.signingBonus',
       'jobPosition.status', 'jobPosition.title', 'jobPosition.startAt',
       'jobPosition.createdAt', 'jobPosition.updatedAt',
       'jobPosition.careerOption', 'jobPosition.jobCategoryIds',
       'jobPosition.period', 'jobPosition.technicalTags',
       'jobPosition.teamTechnicalTags', 'jobPosition.url',
       'jobPosition.isSynced', 'jobPosition.isKdtTheme',
       'jobPosition.company.id', 'jobPosition.company.name',
       'jobPosition.company.address', 'jobPosition.company.serviceName',
       'jobPosition.company.serviceUrl', 'jobPosition.company.homeUrl',
       'jobPosition.company.funding', 'jobPosition.company.hideFunding',
       'jobPosition.company.revenue', 'jobPosition.company.hideRevenue',
       

In [341]:
job_data[0]['job']['detail']['requirements']

'• 비전 모델을 연구/개발하는 리서처와 소통할 수 있는 AI, 컴퓨터 비전 관련 배경지식과 경험을 보유하신 분\n• Label Studio, Airflow를 이용해 데이터 파이프라인, AI QA 프로세스를 구축, 개선해보신 경험이 있으신 분\n• REST API를 활용한 백엔드 서비스 개발 경험이 있으신 분\n• 학습된 딥러닝 모델을 ONNX, TensorRT을 활용하여 최적화 해본 경험이 있으신 분\n• 프로덕션 환경에서 ML 서비스를 운영, 개선한 경험이 있으신 분\n\n\n[ 기술 스택 ]\n• FastAPI, Django, PostgreSQL, kafka\n• Label Studio, MLflow, Deep Lake, Airflow\n• TensorRT, Nvidia Triton Inference Server\n• Docker, Kubernetes\n• AWS, Sagemaker'

In [348]:
job_data[0]['job']['skill_tags'][0]['text']

'Django'

In [350]:
job_data[0]['jobPosition']['description']

'<p>오늘의집은 전 세계 사람들의 라이프스타일을 영감으로 삼아 꿈꾸던 삶을 현실로 만들어주는 서비스입니다. 콘텐츠, 커뮤니티, 그리고 커머스를 통합하여 독창적인 경험을 제공하며, 다양한 콘텐츠와 사용자 간의 소통을 통해 사람들이 자신만의 공간을 디자인하고 삶의 질을 향상시킬 수 있도록 도와주고 있습니다.</p><p><br></p><p>오늘의집페이는 안정적이고 신뢰할 수 있는 결제/정산 서비스를 제공합니다. 또한, 고객이 원하는 다양한 결제수단을 제공하여 더욱 편리한 결제 경험을 제공하고자 합니다. 이를 통해 오늘의집 사용자들이 편리하고 안전하게 거래할 수 있도록 도울 것입니다.</p><p><br></p><p>Software Engineer는 오늘의집 페이를 구성하기 위한 소프트웨어 개발 및 이를 운영하기 위한 제반 인프라와 도구를 개발하는 등 소프트웨어 개발 전반에 걸친 업무를 수행합니다.</p><p><br></p><p><br></p><h3><strong class="gt-strong" style="background-color: rgba(255, 255, 255, 0);">주요업무</strong></h3><ul><li>엔지니어링 역량을 효율적으로 활용하여 오늘의집 페이팀 비즈니스 문제를 해결</li><li>확장성 있는 웹/앱 서비스 및 API의 설계, 구현, 릴리즈</li><li>반응형 디자인을 포함한 애플리케이션 설계 및 구현</li><li>설계 문서 작성 및 테스트 코드 구현 및 버그 수정</li></ul><p><br></p><p><br></p><h3><strong class="gt-strong" style="background-color: rgba(255, 255, 255, 0);">자격요건</strong></h3><ul><li>컴퓨터공학 전공 혹은 그에 준하는 전공 및 지식을 보유한 분</li><li>만 3년 이상의 Software 개발 경력을 보유한 분</li><li>Backend 서버 및 Database(RDB, NoSQL)을 통한 서비스 구현을

In [371]:
df['jobPosition.requirement'].isnull().sum(), df['jobPosition.requirement'].notnull().sum(), len(df)

(np.int64(115), np.int64(621), 736)

In [354]:
url_list

['https://career.programmers.co.kr//job_positions/29477',
 'https://career.programmers.co.kr//job_positions/29476',
 'https://career.programmers.co.kr//job_positions/29475',
 'https://career.programmers.co.kr//job_positions/29474',
 'https://career.programmers.co.kr//job_positions/29290',
 'https://career.programmers.co.kr//job_positions/29472',
 'https://career.programmers.co.kr//job_positions/28747',
 'https://career.programmers.co.kr//job_positions/28294',
 'https://career.programmers.co.kr//job_positions/29431',
 'https://career.programmers.co.kr//job_positions/28411',
 'https://career.programmers.co.kr//job_positions/29393',
 'https://career.programmers.co.kr//job_positions/29470',
 'https://career.programmers.co.kr//job_positions/29469',
 'https://career.programmers.co.kr//job_positions/29426',
 'https://career.programmers.co.kr//job_positions/29339',
 'https://career.programmers.co.kr//job_positions/14672',
 'https://career.programmers.co.kr//job_positions/10687',
 'https://care